# DATA CLEANING AND CONDITIONING

## Introduction

This notebook is dedicated only to the conditioning and cleaning of 6 large spread sheets with basketball stats gathered together from different sources. The files are the following:

* NBA_2000-2010_Game_Results.xlsx

* NBA_2010-2020_Game_Results.xlsx

* 2000-2010_NBA_GEN_STATS.xlsx

* 2010-2020_NBA_GEN_STATS.xlsx

* NBA_2000-2010_AdvStats.xlsx

* NBA_2010_2020_AdvStats.xls

The set of files represent three different databases, each broken down into two files with the intention of separating the last 2 decades and have the possibility to analyze the game evolution during the past 20 years. 

The 'Game_Results' files only contain match-up basic information with game results. From this file I will be generating a column called 'Game_Result' that will have binary values: "1" for Home Team (Team2) winning the game, and "0" for Home Team losing the game. This will be the target ('y') for the models that will be tested. The features will be explained later on this document once the final data-frames are ready. 

The second set of files contains 'Gerenal Statistics'. This are straight forward stats such as FG_P (Field Goal Percentage), 3P_P (three point shooting percentages), among others directly calculated from the game. 

The third data sset corresponds to 'Advanced Statistics'. This are slightly more complicated stats with some calculated from different combinations of the 'General Statistics'.

The objective is to end up with two dataframes with all the information combined and classified to start the modeling. At the end I will be looking only at 9 different combined statistical values for my "Home Team Winning Prediction". These 9 categories will be:

* Offensive Rating: Estimated number of points a team scores per 100 possessions (the higher the better)

* Defensive Rating: Estimated number of points a team gives up per 100 possessions (the lower the better)

* Field Goal Percentage

* Field Goal Attempts

* 3-Point Percentages

* 3-Point Attempts

* Assist/Turnover RAtio

* Rebound Differential

* Pace: Estimated number of possession per 48 minutes (game length)

# Data Loading and QC

To start I will proceed to load the 6 files and look briefly at the first 2 rows to check that everything is in place to then proceed to start creating the binary columns, changing the Date format, and engineering some columns to be able to merge these into a final dataframe per decade

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
df1a = pd.read_excel('NBA_2010-2020_Game_Results.xlsx')
display(df1a.head(2))
display('The shape of df1a is:', df1a.shape)

,Date,Team1,Team1Score,Team2,Team2Score
0,"Tue, Oct 22, 2019",New Orleans Pelicans,122,Toronto Raptors,130
1,"Tue, Oct 22, 2019",Los Angeles Lakers,102,Los Angeles Clippers,112


'The shape of df1a is:'

(12232, 5)

In [3]:
df1b = pd.read_excel('NBA_2000-2010_Game_Results.xlsx')
display(df1b.head(2))
display('The shape of df1b is:', df1b.shape)

,Date,Team1,Team1Score,Team2,Team2Score
0,"Tue, Oct 31, 2000",Charlotte Hornets,106,Atlanta Hawks,82
1,"Tue, Oct 31, 2000",Cleveland Cavaliers,86,New Jersey Nets,82


'The shape of df1b is:'

(12142, 5)

In [4]:
df2a = pd.read_excel('2010-2020_NBA_GEN_STATS.xlsx')
display(df2a.head(2))
display('The shape of df2a is:', df2a.shape)

,TEAM,year,month,PTS,FGM,FGA,FG_P,3PM,3PA,3P_P
0,Atlanta Hawks,2010,10,107.3,37.3,77.0,48.5,6.3,17.3,36.5
1,Los Angeles Lakers,2010,10,111.0,41.7,93.0,44.8,9.0,22.3,40.3


'The shape of df2a is:'

(2048, 10)

In [5]:
df2b = pd.read_excel('2000-2010_NBA_GEN_STATS.xlsx')
display(df2b.head(2))
display('The shape of df2b is:', df2b.shape)

,TEAM,year,month,PTS,FGM,FGA,FG_P,3PM,3PA,3P_P
0,Charlotte Hornets,2000,10,106.0,35.0,62.0,56.5,5.0,9.0,55.6
1,Cleveland Cavaliers,2000,10,86.0,32.0,78.0,41.0,2.0,7.0,28.6


'The shape of df2b is:'

(1973, 10)

In [6]:
df3a = pd.read_excel('NBA_2010-2020_AdvStats.xlsx')
display(df3a.head(2))
display('The shape of df3a is:', df3a.shape)

,TEAM,year,month,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE
0,Miami Heat,2010,10,4,3,1,192,103.6,90.0,13.6,58.9,1.43,15.9,26.0,69.0,49.1,14.7,49.5,54.6,90.00,61.2
1,Dallas Mavericks,2010,10,3,2,1,144,102.5,91.2,11.2,68.5,1.49,19.7,22.8,69.4,48.9,18.0,53.3,56.8,94.67,60.3


'The shape of df3a is:'

(2040, 21)

In [7]:
df3b = pd.read_excel('NBA_2000-2010_AdvStats.xlsx')
display(df3b.head(2))
display('The shape of df3b is:', df3b.shape)

,TEAM,year,month,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE
0,Charlotte Hornets,2000,10,1,1,0,48,114.0,89.1,24.8,45.7,0.94,13.9,23.3,75.5,56.6,18.3,60.5,66.6,92.5,76.6
1,Cleveland Cavaliers,2000,10,1,1,0,48,87.8,83.7,4.1,50.0,0.84,12.4,30.2,72.4,52.3,19.4,42.3,46.7,98.0,51.2


'The shape of df3b is:'

(2183, 21)

# Dataframe Pre-processing

## Game Results DataFrames 

### 2010 - 2020 seasons

In [8]:
# Converting the 'Date' column using the Pandas DatetimeIndex to extract the year and month as new columns
df1a['year'] = pd.DatetimeIndex(df1a['Date']).year
df1a['month'] = pd.DatetimeIndex(df1a['Date']).month

# Dropping the 'Date' column as it won't be necessary anymore 
df1a.drop(['Date'], axis=1, inplace=True)

# Creating a binary column from the game results
df1a['Game_Result'] = np.where(df1a['Team2Score'] > df1a['Team1Score'], 1, 0)

print('The shape of the dataframe is:', df1a.shape)
print(' ')
display(df1a.head(2))
print('=====================================================================================')
print("Results of the Game_Result binnary column are:")
print(df1a.Game_Result.value_counts())

The shape of the dataframe is: (12232, 7)
 


,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result
0,New Orleans Pelicans,122,Toronto Raptors,130,2019,10,1
1,Los Angeles Lakers,102,Los Angeles Clippers,112,2019,10,1


Results of the Game_Result binnary column are:
1    7181
0    5051
Name: Game_Result, dtype: int64


In average, 52.63% of the games are won by the home team (Team2). This is not that surprising as not all the teams are strong enough to protect home court advantage and historically there has been a tier of NBA teams that have been weaker compared to tier 2 and tier 1 teams. The reasons for this have been mainly two, first the team's location is not attractive for players, and second, the team doesn't have enough cash to afford good players. 

Lets see the previous decade...

### 2000 - 2010 seasons

In [9]:
# Converting the 'Date' column using the Pandas DatetimeIndex to extract the year and month as new columns
df1b['year'] = pd.DatetimeIndex(df1b['Date']).year
df1b['month'] = pd.DatetimeIndex(df1b['Date']).month

# Dropping the 'Date' column as it won't be necessary anymore 
df1b.drop(['Date'], axis=1, inplace=True)

# Creating a binary column from the game results
df1b['Game_Result'] = np.where(df1b['Team2Score'] > df1b['Team1Score'], 1, 0)

print('The shape of the dataframe is:', df1b.shape)
print('=====================================================================================')
display(df1b.head(2))
print('=====================================================================================')
print("Results of the Game_Result binnary column are:")
print(df1b.Game_Result.value_counts())

The shape of the dataframe is: (12142, 7)


,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result
0,Charlotte Hornets,106,Atlanta Hawks,82,2000,10,0
1,Cleveland Cavaliers,86,New Jersey Nets,82,2000,10,0


Results of the Game_Result binnary column are:
1    7323
0    4819
Name: Game_Result, dtype: int64


In [10]:
df1b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12142 entries, 0 to 12141
Data columns (total 7 columns):
Team1          12142 non-null object
Team1Score     12142 non-null int64
Team2          12142 non-null object
Team2Score     12142 non-null int64
year           12142 non-null int64
month          12142 non-null int64
Game_Result    12142 non-null int64
dtypes: int64(5), object(2)
memory usage: 664.1+ KB


During the previous decade an average of 60.31% of the games were won by the home team. It is known that on that decade there was a bit more parity between teams. Still there was a tier 3 group of teams which were relatively weak compared to the Tier 2 and in particular the Tier 1 teams. 

There are a couple of columns with dtypes equal to 'object' but it's alright as those will be dropped later on after the final merge and from these two df I will only keep the 'target' which is the Game_Result

## General & Advance Statistics DataFrames

Before I go into creating my final DataFrame per decade I need to merge df2a with df3a and df2b with df3b. For this to be done properly I will engineer a temporary column with I will call Team_id in each of the 4 DataFrames. Team_id will correspond to the Team's Name, the month and the year. The idea here is that I can then assign the average values of each of these stats to the df1a and df1b.

In [11]:
display(df2a.head(2))
display("The size of df2a is:", df2a.shape)
display(df3a.head(2))
display("The size of this df3a is:", df3a.shape)

,TEAM,year,month,PTS,FGM,FGA,FG_P,3PM,3PA,3P_P
0,Atlanta Hawks,2010,10,107.3,37.3,77.0,48.5,6.3,17.3,36.5
1,Los Angeles Lakers,2010,10,111.0,41.7,93.0,44.8,9.0,22.3,40.3


'The size of df2a is:'

(2048, 10)

,TEAM,year,month,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE
0,Miami Heat,2010,10,4,3,1,192,103.6,90.0,13.6,58.9,1.43,15.9,26.0,69.0,49.1,14.7,49.5,54.6,90.00,61.2
1,Dallas Mavericks,2010,10,3,2,1,144,102.5,91.2,11.2,68.5,1.49,19.7,22.8,69.4,48.9,18.0,53.3,56.8,94.67,60.3


'The size of this df3a is:'

(2040, 21)

In [12]:
# For the 4 different Dataframes:
df2a['Team_id'] = df2a['TEAM'].astype(str)+'_'+df2a['month'].astype(str)+'_'+df2a['year'].astype(str)
df2b['Team_id'] = df2b['TEAM'].astype(str)+'_'+df2b['month'].astype(str)+'_'+df2b['year'].astype(str)
df3a['Team_id'] = df3a['TEAM'].astype(str)+'_'+df3a['month'].astype(str)+'_'+df3a['year'].astype(str)
df3b['Team_id'] = df3b['TEAM'].astype(str)+'_'+df3b['month'].astype(str)+'_'+df3b['year'].astype(str)

Now, look at the new columns called 'Team_id' on the two display dataframes below. They should have the name of the team with the month and year for the average values from each of the previous two DataFrames

In [13]:
display(df2a.head(1))
display(df3a.head(1))

,TEAM,year,month,PTS,FGM,FGA,FG_P,3PM,3PA,3P_P,Team_id
0,Atlanta Hawks,2010,10,107.3,37.3,77.0,48.5,6.3,17.3,36.5,Atlanta Hawks_10_2010


,TEAM,year,month,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE,Team_id
0,Miami Heat,2010,10,4,3,1,192,103.6,90.0,13.6,58.9,1.43,15.9,26.0,69.0,49.1,14.7,49.5,54.6,90.0,61.2,Miami Heat_10_2010


In [14]:
# First lets take care of df2a with df3a
df2 = df2a.merge(df3a, on="Team_id", how = 'inner')
df2 = df2.drop(['PTS','FGM', '3PM', 'TEAM_y', 'year_y', 'month_y', 'MIN'], axis=1)
df2.rename({'TEAM_x': 'TEAM', 'year_x': 'year', 'month_x': 'month'}, axis=1, inplace=True)
print('DataFrame df2:')
display(df2.head(1).append(df2.tail(1)))

# Now lets take care of df2b with df3b
df3 = df2b.merge(df3b, on="Team_id", how = 'inner')
df3 = df3.drop(['PTS','FGM', '3PM', 'TEAM_y', 'year_y', 'month_y', 'MIN'], axis=1)
df3.rename({'TEAM_x': 'TEAM', 'year_x': 'year', 'month_x': 'month'}, axis=1, inplace=True)
print('DataFrame df3:')
display(df3.head(1).append(df3.tail(1)))

DataFrame df2:


,TEAM,year,month,FGA,FG_P,3PA,3P_P,Team_id,GP,W,L,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE
0,Atlanta Hawks,2010,10,77.0,48.5,17.3,36.5,Atlanta Hawks_10_2010,3,3,0,112.2,104.5,7.7,56.3,1.21,16.2,32.0,72.2,53.5,18.1,52.6,59.3,95.67,55.7
2040,Washington Wizards,2020,8,90.3,42.1,29.4,33.0,Washington Wizards_8_2020,7,1,6,102.4,111.4,-9.0,60.9,1.59,16.5,28.1,74.0,49.9,14.4,47.5,51.7,101.43,44.4


DataFrame df3:


,TEAM,year,month,FGA,FG_P,3PA,3P_P,Team_id,GP,W,L,OFFRTG,DEFRTG,NETRTG,AST_P,AST/TO,ASTRATIO,OREB_P,DREB_P,REB_P,TOV_P,EFG_P,TS_P,PACE,PIE
0,Charlotte Hornets,2000,10,62.0,56.5,9.0,55.6,Charlotte Hornets_10_2000,1,1,0,114.0,89.1,24.8,45.7,0.94,13.9,23.3,75.5,56.6,18.3,60.5,66.6,92.50,76.6
2178,Minnesota Timberwolves,2010,4,78.4,45.7,17.0,35.3,Minnesota Timberwolves_4_2010,7,0,7,100.7,114.5,-13.7,57.4,1.25,16.1,25.0,68.2,46.1,17.0,49.5,54.2,96.79,41.7


In [15]:
# Once again I'll deal first with df2 creating a new and reduced DataFramee called df2_ready
df2_ready = df2[['TEAM', 'Team_id', 'year', 'month', 'FGA', 'FG_P', '3PA', '3P_P', 'OFFRTG', 'DEFRTG', 'REB_P', 'AST/TO', 'PACE']]
print('DataFrame df2_ready:')
display(df2_ready.head(3).append(df2_ready.tail(3)))

# And now I'll deal with df3 creating as well a new summarized df3_ready:
df3_ready = df3[['TEAM', 'Team_id', 'year', 'month', 'FGA', 'FG_P', '3PA', '3P_P', 'OFFRTG', 'DEFRTG', 'REB_P', 'AST/TO', 'PACE']]
print('DataFrame df3_ready:')
display(df3_ready.head(3).append(df3_ready.tail(3)))

DataFrame df2_ready:


,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Atlanta Hawks,Atlanta Hawks_10_2010,2010,10,77.0,48.5,17.3,36.5,112.2,104.5,53.5,1.21,95.67
1,Los Angeles Lakers,Los Angeles Lakers_10_2010,2010,10,93.0,44.8,22.3,40.3,112.1,101.0,51.6,1.56,98.83
2,New Orleans Hornets,New Orleans Hornets_10_2010,2010,10,78.0,47.4,15.0,37.8,104.2,97.5,48.9,1.65,94.33
2038,Orlando Magic,Orlando Magic_8_2020,2020,8,86.9,45.2,34.7,35.4,110.0,113.0,50.5,1.52,100.93
2039,Utah Jazz,Utah Jazz_8_2020,2020,8,90.4,44.2,43.3,37.0,110.2,114.5,50.8,1.56,100.58
2040,Washington Wizards,Washington Wizards_8_2020,2020,8,90.3,42.1,29.4,33.0,102.4,111.4,49.9,1.59,101.43


DataFrame df3_ready:


,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Charlotte Hornets,Charlotte Hornets_10_2000,2000,10,62.0,56.5,9.0,55.6,114.0,89.1,56.6,0.94,92.50
1,Cleveland Cavaliers,Cleveland Cavaliers_10_2000,2000,10,78.0,41.0,7.0,28.6,87.8,83.7,52.3,0.84,98.00
2,Dallas Mavericks,Dallas Mavericks_10_2000,2000,10,88.0,39.8,25.0,36.0,99.0,93.9,51.4,1.22,98.50
2176,Sacramento Kings,Sacramento Kings_4_2010,2010,4,85.0,48.0,12.8,35.1,108.8,115.2,51.3,1.60,92.25
2177,Philadelphia 76ers,Philadelphia 76ers_4_2010,2010,4,79.3,52.4,21.6,41.1,114.4,118.9,47.5,1.95,92.06
2178,Minnesota Timberwolves,Minnesota Timberwolves_4_2010,2010,4,78.4,45.7,17.0,35.3,100.7,114.5,46.1,1.25,96.79


Just to summarize what I have done, so far we have cleaned and conditioned the three dataframes that were initially broken down into two each: df1a, df1b, df2a, df2b, df3a, and df3b. These Dataframes were separated by decades: from the year 2000 to 2010, and from the year 2010 to 2020. The idea is at the end to have just two DataFrames, one per decade with the selected categories (columns) to be used in the modeling for our predictions. Still I will have available the a considerably larger couple of Dataframes with the rest of the data just in case that any other column/category/stat wants to be added to the models. 

Remember that these stats columns were chosen following what the NBA Analysts used today to predict game results. However, as explained before, there are a lot of other factors that can contribute to an upset and those unfortunately are not accounted for on my models, therefore there will be a percentage of error on my predictions which I am expecting to be circa 30% which is more less what the NBA has.

The next step is now to add these values to the initial DataFrame with the Game_Results (df1a and df1b) and then the data is model ready! So I will save at this point each DataFrame as separate .csv files and then proceed for the final Dataframes to be Model Ready...

In [16]:
df2_ready.to_csv('2010-2020_Monthly_Team_9MStats.csv')
df3_ready.to_csv('2000-2010_Monthly_Team_9MStats.csv')

# Building Model Ready DataFrames

In [17]:
results_a = df1a
display(results_a.head(2))
results_b = df1b
display(results_b.head(2))

,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result
0,New Orleans Pelicans,122,Toronto Raptors,130,2019,10,1
1,Los Angeles Lakers,102,Los Angeles Clippers,112,2019,10,1


,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result
0,Charlotte Hornets,106,Atlanta Hawks,82,2000,10,0
1,Cleveland Cavaliers,86,New Jersey Nets,82,2000,10,0


In [18]:
stats_a = df2_ready
display(stats_a.head(1))
stats_b = df3_ready
display(stats_b.head(1))

,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Atlanta Hawks,Atlanta Hawks_10_2010,2010,10,77.0,48.5,17.3,36.5,112.2,104.5,53.5,1.21,95.67


,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Charlotte Hornets,Charlotte Hornets_10_2000,2000,10,62.0,56.5,9.0,55.6,114.0,89.1,56.6,0.94,92.5


Now, in order to merge these tables I will need to create new columns as I previously did for the initial merge. In that initial merge that I have now called stats_a and stats_b respectively, I preserved that Team_id column, so I will only need to make new ones for the results_a and results_b DataFrame.

In [19]:
# Engineering new columns for results_a DataFrame
results_a['Team1_id'] = results_a['Team1'].astype(str)+'_'+results_a['month'].astype(str)+'_'+results_a['year'].astype(str)
results_a['Team2_id'] = results_a['Team2'].astype(str)+'_'+results_a['month'].astype(str)+'_'+results_a['year'].astype(str)

#Engineering new columns for results_b DataFrame
results_b['Team1_id'] = results_b['Team1'].astype(str)+'_'+results_b['month'].astype(str)+'_'+results_b['year'].astype(str)
results_b['Team2_id'] = results_b['Team2'].astype(str)+'_'+results_b['month'].astype(str)+'_'+results_b['year'].astype(str)

We are all set for our final merge, but before a last fast QC by looking at the possible missing values, the first two rows of each DataFrames and their respectives shapes:

In [20]:
print("Total number of missing values for each DataFrame:")
print("=================================================")
print("results_a:", results_a.isna().sum().sum())
print("results_b:", results_b.isna().sum().sum())
print("stats_a:", stats_a.isna().sum().sum())
print("stats_b:", stats_b.isna().sum().sum())

Total number of missing values for each DataFrame:
results_a: 0
results_b: 0
stats_a: 0
stats_b: 0


In [21]:
# Displays and shapes of the results_a and stats_a before being merged together:
display(results_a.shape)
display(results_a.head(2))
display(stats_a.shape)
display(stats_a.head(2))

# Displays and shapes of the results_b and stats_b before being merged together
display(results_b.shape)
display(results_b.head(2))
display(stats_b.shape)
display(stats_b.head(2))

(12232, 9)

,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result,Team1_id,Team2_id
0,New Orleans Pelicans,122,Toronto Raptors,130,2019,10,1,New Orleans Pelicans_10_2019,Toronto Raptors_10_2019
1,Los Angeles Lakers,102,Los Angeles Clippers,112,2019,10,1,Los Angeles Lakers_10_2019,Los Angeles Clippers_10_2019


(2041, 13)

,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Atlanta Hawks,Atlanta Hawks_10_2010,2010,10,77.0,48.5,17.3,36.5,112.2,104.5,53.5,1.21,95.67
1,Los Angeles Lakers,Los Angeles Lakers_10_2010,2010,10,93.0,44.8,22.3,40.3,112.1,101.0,51.6,1.56,98.83


(12142, 9)

,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result,Team1_id,Team2_id
0,Charlotte Hornets,106,Atlanta Hawks,82,2000,10,0,Charlotte Hornets_10_2000,Atlanta Hawks_10_2000
1,Cleveland Cavaliers,86,New Jersey Nets,82,2000,10,0,Cleveland Cavaliers_10_2000,New Jersey Nets_10_2000


(2179, 13)

,TEAM,Team_id,year,month,FGA,FG_P,3PA,3P_P,OFFRTG,DEFRTG,REB_P,AST/TO,PACE
0,Charlotte Hornets,Charlotte Hornets_10_2000,2000,10,62.0,56.5,9.0,55.6,114.0,89.1,56.6,0.94,92.5
1,Cleveland Cavaliers,Cleveland Cavaliers_10_2000,2000,10,78.0,41.0,7.0,28.6,87.8,83.7,52.3,0.84,98.0


Now by displaying the DataFramess yo can see the similarities between the TEAM_id, Team1_id, and Team2_id columns. In fact they have exactly the same content in terms of games and dates. I can imagine that now it is easier for the reader to understand the workflow, as you can picture how the merges will take place using those columns

Now to the most important part of the process...merging the DataFrames to have one model ready output per decade:

In [22]:
# Merge for the results_a and stats_a pair:
df_merge_a = results_a.merge(
    stats_a.add_prefix('Team1_'), left_on='Team1_id', right_on='Team1_Team_id',how='left').merge(
    stats_a.add_prefix('Team2_'), left_on='Team2_id', right_on='Team2_Team_id',how='left').drop(
    columns= ['Team1_id','Team2_id','Team1_Team_id','Team2_Team_id','Team1_TEAM','Team2_TEAM'])

In [23]:
# Merge for the results_b and stats_b pair:
df_merge_b = results_b.merge(
    stats_b.add_prefix('Team1_'), left_on='Team1_id', right_on='Team1_Team_id',how='left').merge(
    stats_b.add_prefix('Team2_'), left_on='Team2_id', right_on='Team2_Team_id',how='left').drop(
    columns= ['Team1_id','Team2_id','Team1_Team_id','Team2_Team_id','Team1_TEAM','Team2_TEAM'])

Both merged DataFrames are ready, so lets have a quick look at them:

In [24]:
print("df_merge_a")
display(df_merge_a.head())
print("Shape of df_merge_a:", df_merge_a.shape)
print(' ')
print('======================================================================================')
print(' ')
print("df_merge_b")
display(df_merge_b.head())
print("Shape of df_merge_b:", df_merge_b.shape)

df_merge_a


,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result,Team1_year,Team1_month,Team1_FGA,Team1_FG_P,Team1_3PA,Team1_3P_P,Team1_OFFRTG,Team1_DEFRTG,Team1_REB_P,Team1_AST/TO,Team1_PACE,Team2_year,Team2_month,Team2_FGA,Team2_FG_P,Team2_3PA,Team2_3P_P,Team2_OFFRTG,Team2_DEFRTG,Team2_REB_P,Team2_AST/TO,Team2_PACE
0,New Orleans Pelicans,122,Toronto Raptors,130,2019,10,1,2019.0,10.0,96.0,46.5,43.2,38.0,112.2,114.2,46.9,1.89,106.09,2019.0,10.0,86.8,46.3,36.8,38.0,107.5,99.1,51.1,1.26,104.23
1,Los Angeles Lakers,102,Los Angeles Clippers,112,2019,10,1,2019.0,10.0,87.5,45.7,29.0,33.6,107.9,96.8,51.4,1.54,101.00,2019.0,10.0,84.0,49.6,29.5,36.7,112.9,107.5,49.6,1.35,101.50
2,Chicago Bulls,125,Charlotte Hornets,126,2019,10,1,2019.0,10.0,94.0,41.1,36.2,29.3,103.7,109.6,45.2,1.81,101.80,2019.0,10.0,86.0,46.3,35.8,41.9,104.9,114.2,47.3,1.30,103.00
3,Detroit Pistons,119,Indiana Pacers,110,2019,10,0,2019.0,10.0,81.6,47.8,26.8,41.8,106.3,110.4,50.9,1.23,101.70,2019.0,10.0,91.5,44.8,24.0,36.5,105.3,108.5,45.8,2.00,99.88
4,Cleveland Cavaliers,85,Orlando Magic,94,2019,10,1,2019.0,10.0,86.5,46.0,32.0,32.8,105.5,107.7,53.1,1.32,100.50,2019.0,10.0,90.5,39.5,33.0,28.8,97.2,94.7,49.2,1.77,98.75


Shape of df_merge_a: (12233, 29)
 
 
df_merge_b


,Team1,Team1Score,Team2,Team2Score,year,month,Game_Result,Team1_year,Team1_month,Team1_FGA,Team1_FG_P,Team1_3PA,Team1_3P_P,Team1_OFFRTG,Team1_DEFRTG,Team1_REB_P,Team1_AST/TO,Team1_PACE,Team2_year,Team2_month,Team2_FGA,Team2_FG_P,Team2_3PA,Team2_3P_P,Team2_OFFRTG,Team2_DEFRTG,Team2_REB_P,Team2_AST/TO,Team2_PACE
0,Charlotte Hornets,106,Atlanta Hawks,82,2000,10,0,2000.0,10.0,62.0,56.5,9.0,55.6,114.0,89.1,56.6,0.94,92.5,2000.0,10.0,81.0,37.0,15.0,40.0,89.1,114.0,43.4,1.08,92.5
1,Cleveland Cavaliers,86,New Jersey Nets,82,2000,10,0,2000.0,10.0,78.0,41.0,7.0,28.6,87.8,83.7,52.3,0.84,98.0,2000.0,10.0,85.0,36.5,10.0,30.0,83.7,87.8,47.7,1.60,98.0
2,Washington Wizards,86,Orlando Magic,97,2000,10,1,2000.0,10.0,72.0,45.8,7.0,57.1,87.8,99.0,53.3,0.74,98.0,2000.0,10.0,79.0,43.0,16.0,37.5,99.0,87.8,46.7,1.33,98.0
3,Milwaukee Bucks,93,Dallas Mavericks,97,2000,10,1,2000.0,10.0,83.0,39.8,19.0,31.6,93.9,99.0,48.6,0.80,98.5,2000.0,10.0,88.0,39.8,25.0,36.0,99.0,93.9,51.4,1.22,98.5
4,Philadelphia 76ers,101,New York Knicks,72,2000,10,0,2000.0,10.0,66.0,57.6,8.0,37.5,114.8,81.8,49.4,1.93,88.0,2000.0,10.0,70.0,35.7,11.0,27.3,81.8,114.8,50.6,0.64,88.0


Shape of df_merge_b: (15808, 29)


We are almost ready to save these DataFrames as csv files, but before I do that, I need to check for missing values. The NBA has gone through a few transformations by changing the names of a few teams that have been moved from one city to another, or have changed owners. There is also the problem of team name abbreviation such as LA Clippers instead of Los Angeles Clippers, and the same case for LA Lakers. This particular case was already corrected but others are hard to find as the final DataFrames are rather large.

Let's have a quick look for some of those missing values and see now what numbers we are dealing with:

In [25]:
display(df_merge_a.isna().sum())
display(df_merge_a.shape)
display(df_merge_b.isna().sum())
display(df_merge_b.shape)

Team1            0
Team1Score       0
Team2            0
Team2Score       0
year             0
month            0
Game_Result      0
Team1_year      35
Team1_month     35
Team1_FGA       35
Team1_FG_P      35
Team1_3PA       35
Team1_3P_P      35
Team1_OFFRTG    35
Team1_DEFRTG    35
Team1_REB_P     35
Team1_AST/TO    35
Team1_PACE      35
Team2_year      35
Team2_month     35
Team2_FGA       35
Team2_FG_P      35
Team2_3PA       35
Team2_3P_P      35
Team2_OFFRTG    35
Team2_DEFRTG    35
Team2_REB_P     35
Team2_AST/TO    35
Team2_PACE      35
dtype: int64

(12233, 29)

Team1            0
Team1Score       0
Team2            0
Team2Score       0
year             0
month            0
Game_Result      0
Team1_year      14
Team1_month     14
Team1_FGA       14
Team1_FG_P      14
Team1_3PA       14
Team1_3P_P      14
Team1_OFFRTG    14
Team1_DEFRTG    14
Team1_REB_P     14
Team1_AST/TO    14
Team1_PACE      14
Team2_year      12
Team2_month     12
Team2_FGA       12
Team2_FG_P      12
Team2_3PA       12
Team2_3P_P      12
Team2_OFFRTG    12
Team2_DEFRTG    12
Team2_REB_P     12
Team2_AST/TO    12
Team2_PACE      12
dtype: int64

(15808, 29)

The first DataFrame (df_merge_a) has 35 rows of missing values out of 12233. This represents 0.28%, so considerably less than 1%. The decision here is very easy and that is that I will drop those rows. 

For the second DataFrame (df_merge_b) it's 24 different rows that have missing values. That accountss for 0.15%, so even a lower percentage than for the first DataFrame, so even an easier decision here. I will then proceed to drop all of those rows, drop the columns that I am not interested in and saveed .csv files for each one: 

In [26]:
# Dropping the rows with missing values for df_merge_a and df_merge_b:
df_merge_a = df_merge_a.dropna()
df_merge_b = df_merge_b.dropna()

df_merge_a.to_csv('df_merge_a.csv')
df_merge_b.to_csv('df_merge_b.csv')

# Dropping the rows which I am not interested in for the modeling and renaming those DataFrames:
df_model_a = df_merge_a.drop(['Team1', 'Team1Score', 'Team2', 'Team2Score', 'year', 'month', 'Team1_month', 'Team2_month', 'Team1_year', 'Team2_year'], axis=1)
df_model_b = df_merge_b.drop(['Team1', 'Team1Score', 'Team2', 'Team2Score', 'year', 'month', 'Team1_month', 'Team2_month', 'Team1_year', 'Team2_year'], axis=1)

=======================================================================================================

In [27]:
# Finally saving both DataFrames as .csv files for the EDA and modeling

df_model_a.to_csv('2010-2020_Model_Ready.csv')
df_model_b.to_csv('2000-2010_Model_Ready.csv')

=======================================================================================================